In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
from tensorflow.keras.optimizers import Adam

def densenet(input_shape, n_classes, filters=32):
    def bn_rl_conv(x, filters, kernel=1, strides=1):
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides, padding='same')(x)
        return x

    def dense_block(x, repetitions):
        for _ in range(repetitions):
            y = bn_rl_conv(x, 4 * filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y, x])
        return x

    def transition_layer(x):
        x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
        x = AvgPool2D(2, strides=2, padding='same')(x)
        return x

    input_layer = Input(input_shape)
    x = Conv2D(64, 3, strides=1, padding='same')(input_layer)  # Adjusted for 48x48 input
    x = MaxPool2D(3, strides=2, padding='same')(x)

    for repetitions in [6, 12, 24, 16]:
        d = dense_block(x, repetitions)
        x = transition_layer(d)
    
    x = GlobalAveragePooling2D()(x)
    output = Dense(n_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output)
    return model

# Setup paths
train_dir = '/home/darksst/Desktop/FinalProject/project_1_dataset/train'
test_dir = '/home/darksst/Desktop/FinalProject/project_1_dataset/test'

# Data generators with augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical')

# Create the model
model = densenet((48, 48, 1), 7)  # 7 classes
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=100, validation_data=test_generator)


Found 22939 images belonging to 7 classes.
Found 5726 images belonging to 7 classes.
Epoch 1/100
717/717 [==============================] - 75s 59ms/step - loss: 1.8645 - accuracy: 0.2443 - val_loss: 4.0449 - val_accuracy: 0.1568
Epoch 2/100
717/717 [==============================] - 42s 58ms/step - loss: 1.7398 - accuracy: 0.2866 - val_loss: 2.1348 - val_accuracy: 0.1842
Epoch 3/100
717/717 [==============================] - 43s 60ms/step - loss: 1.5796 - accuracy: 0.3750 - val_loss: 2.0288 - val_accuracy: 0.3161
Epoch 4/100
717/717 [==============================] - 42s 59ms/step - loss: 1.4602 - accuracy: 0.4314 - val_loss: 2.4483 - val_accuracy: 0.2875
Epoch 5/100
717/717 [==============================] - 41s 58ms/step - loss: 1.3604 - accuracy: 0.4779 - val_loss: 2.4774 - val_accuracy: 0.3166
Epoch 6/100
717/717 [==============================] - 42s 59ms/step - loss: 1.2839 - accuracy: 0.5106 - val_loss: 1.3746 - val_accuracy: 0.4820
Epoch 7/100
717/717 [========================

In [12]:
NewDenseK = '/home/darksst/Desktop/FacialExpressionRecognitionUsingDensenet121/CDense.keras'
NewDense = '/home/darksst/Desktop/FacialExpressionRecognitionUsingDensenet121/CDense'
model.save(NewDense)
model.save(NewDenseK, save_format='keras')

INFO:tensorflow:Assets written to: /home/darksst/Desktop/FacialExpressionRecognitionUsingDensenet121/CDense/assets


INFO:tensorflow:Assets written to: /home/darksst/Desktop/FacialExpressionRecognitionUsingDensenet121/CDense/assets
